In [1]:
import plotly
from plotly import graph_objs
from IPython.display import Image
from IPython.display import display, HTML
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier as XGBoostClassifier
from TwitterFunc import *
from wordCloud import *
from collections import Counter
import nltk
import pandas as pd
from time import time
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import numpy as np
import csv

#import plotly.plotly as py 
#import plotly.tools as tls 
#import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



In [2]:
import random
seed = 666
random.seed(seed)

In [3]:
auto_open = False
do_BernoulliNB = False
do_RandomForest = False
do_RandomForest_full_model = True
do_XGBoost = True
plot_all = True

In [4]:

data = TwitterData_Initialize()
data.initialize("data/train.txt", is_spain = False)

#initial data 
data.processed_data.head(10)

username                 date  retweets  favorites  \
0        kabrota  2017-09-24 23:59:12         1          1   
1       najarumi  2017-09-24 23:56:39         1          2   
2  MerceMrsDeath  2017-09-24 23:55:36         0          0   
3    nohopararan  2017-09-25 23:50:06         0          1   
4       OsamaPls  2017-09-25 23:45:55         0          0   
5      breifjart  2017-09-25 23:40:22         0          0   
6      MIQUEL_01  2017-09-25 23:36:39         0          2   
7        r_wicca  2017-09-25 23:33:48         0          0   
8  rafavilaclara  2017-09-26 23:57:59         0          0   
9      ateixidom  2017-09-26 23:51:27         0          0   

                                                text  \
0  Els del Piolin estan que mosseguen. El sopar q...   
1  Q gràn la guàrdia civil d'Expaña internacional...   
2  Però a veure, k mda més manestuós i elegant a ...   
3  #freepiolin detencions, portavions#ref1o no ar...   
4  A jurar piolínes! #FreePiolin https://twitter....   
5  Qualsevol dia a una ciutat d'Espanya... #FreeP...   
6  LAS URNAS LAS CARGO EL DIABLO........EL DIA QU...   
7  Quin nivell Entre el #FreePiolin i @albert_pla...   
8  És al·lucinant! Sóm una colònia! Començo a pen...   
9  @mariusserra insultos??? No som nosaltres qui ...   

                                            emoticon location      mentions  \
0                                                 []      NaN           NaN   
1  ["'\\U0001f425'", "'\\U0001f425'", "'\\U0001f4...      NaN           NaN   
2                                                 []      NaN           NaN   
3                                                 []      NaN           NaN   
4                                                 []      NaN           NaN   
5                                                 []      NaN           NaN   
6                                                 []      NaN           NaN   
7  ["'\\U0001f923'", "'\\U0001f923'", "'\\U0001f9...      NaN   @albert_pla   
8                                                 []      NaN           NaN   
9                                                 []      NaN  @mariusserra   

                                             hashtag                  id  \
0                                        #FreePiolin  912104160069345280   
1                               #FreePiolin #Votarem  912103517011283968   
2                                        #FreePiolin  912103254921814016   
3                                 #freepiolin #ref1o  912464257526243330   
4                                        #FreePiolin  912463203074035712   
5         #FreePiolin #1OCTReferendum #Cataloniahttp  912461809310949376   
6  #LoQueLaUrnaEsconde #FreePiolín #HolaDictadurapic  912460873595269120   
7                                        #FreePiolin  912460157619179521   
8                                        #FreePiolin  912828629465300992   
9                                        #FreePiolin  912826985373732865   

                                           permalink language coordinates  \
0  https://twitter.com/kabrota/status/91210416006...       ca         NaN   
1  https://twitter.com/najarumi/status/9121035170...       ca         NaN   
2  https://twitter.com/MerceMrsDeath/status/91210...       ca         NaN   
3  https://twitter.com/nohopararan/status/9124642...       ca         NaN   
4  https://twitter.com/OsamaPls/status/9124632030...       ca         NaN   
5  https://twitter.com/breifjart/status/912461809...       ca         NaN   
6  https://twitter.com/MIQUEL_01/status/912460873...       ca         NaN   
7  https://twitter.com/r_wicca/status/91246015761...       ca         NaN   
8  https://twitter.com/rafavilaclara/status/91282...       ca         NaN   
9  https://twitter.com/ateixidom/status/912826985...       ca         NaN   

  sentiment  
0  positive  
1  positive  
2  positive  
3  positive  
4  positive  
5  positive  
6  positive  
7  positive  
8  positive  
9  positive

In [5]:
'''
First thing that can be done as soon as the data is loaded is to see the 
data distribution. The training set had the following distribution
'''

if plot_all:
    df = data.processed_data
    positive = len(df[df["sentiment"] == "positive"])
    negative = len(df[df["sentiment"] == "negative"])
    neutral = len(df[df["sentiment"] == "neutral"])
    dist = [
    graph_objs.Bar(
        x=['positive','negative','neutral'],
        y= [positive,negative, neutral],
    )]

    plotly.offline.iplot({
    "data":dist, 
    "layout":graph_objs.Layout(title="Sentiment type distribution in training set")})

    

In [6]:
'''
Cleansing

For the purpose of cleansing, the TwitterCleanup class was created. 
It consists methods allowing to execute all of the tasks show in the list above. 
Most of those is done using regular expressions.
The class exposes it's interface through iterate() method - it yields every cleanup 
method in proper order.
'''
data = TwitterData_Cleansing(data)
data.cleanup(TwitterCleanuper())

data.processed_data.head(10)

username                 date  retweets  favorites  \
0        kabrota  2017-09-24 23:59:12         1          1   
1       najarumi  2017-09-24 23:56:39         1          2   
2  MerceMrsDeath  2017-09-24 23:55:36         0          0   
3    nohopararan  2017-09-25 23:50:06         0          1   
4       OsamaPls  2017-09-25 23:45:55         0          0   
5      breifjart  2017-09-25 23:40:22         0          0   
6      MIQUEL_01  2017-09-25 23:36:39         0          2   
7        r_wicca  2017-09-25 23:33:48         0          0   
8  rafavilaclara  2017-09-26 23:57:59         0          0   
9      ateixidom  2017-09-26 23:51:27         0          0   

                                                text  \
0  Els del Piolin estan que mosseguen El sopar qu...   
1  Q gràn la guàrdia civil dExpaña internacionali...   
2  Però a veure k mda més manestuós i elegant a u...   
3  freepiolin detencions portavionsrefo no arross...   
4                      A jurar piolínes FreePiolin     
5  Qualsevol dia a una ciutat dEspanya FreePiolin...   
6  LAS URNAS LAS CARGO EL DIABLOEL DIA QUE LA CAG...   
7  Quin nivell Entre el FreePiolin i ja em puc mo...   
8  És al·lucinant Sóm una colònia Començo a pensa...   
9  insultos No som nosaltres qui hem fletat un ba...   

                                            emoticon location      mentions  \
0                                                 []      NaN           NaN   
1  ["'\\U0001f425'", "'\\U0001f425'", "'\\U0001f4...      NaN           NaN   
2                                                 []      NaN           NaN   
3                                                 []      NaN           NaN   
4                                                 []      NaN           NaN   
5                                                 []      NaN           NaN   
6                                                 []      NaN           NaN   
7  ["'\\U0001f923'", "'\\U0001f923'", "'\\U0001f9...      NaN   @albert_pla   
8                                                 []      NaN           NaN   
9                                                 []      NaN  @mariusserra   

                                             hashtag                  id  \
0                                        #FreePiolin  912104160069345280   
1                               #FreePiolin #Votarem  912103517011283968   
2                                        #FreePiolin  912103254921814016   
3                                 #freepiolin #ref1o  912464257526243330   
4                                        #FreePiolin  912463203074035712   
5         #FreePiolin #1OCTReferendum #Cataloniahttp  912461809310949376   
6  #LoQueLaUrnaEsconde #FreePiolín #HolaDictadurapic  912460873595269120   
7                                        #FreePiolin  912460157619179521   
8                                        #FreePiolin  912828629465300992   
9                                        #FreePiolin  912826985373732865   

                                           permalink language coordinates  \
0  https://twitter.com/kabrota/status/91210416006...       ca         NaN   
1  https://twitter.com/najarumi/status/9121035170...       ca         NaN   
2  https://twitter.com/MerceMrsDeath/status/91210...       ca         NaN   
3  https://twitter.com/nohopararan/status/9124642...       ca         NaN   
4  https://twitter.com/OsamaPls/status/9124632030...       ca         NaN   
5  https://twitter.com/breifjart/status/912461809...       ca         NaN   
6  https://twitter.com/MIQUEL_01/status/912460873...       ca         NaN   
7  https://twitter.com/r_wicca/status/91246015761...       ca         NaN   
8  https://twitter.com/rafavilaclara/status/91282...       ca         NaN   
9  https://twitter.com/ateixidom/status/912826985...       ca         NaN   

  sentiment  
0  positive  
1  positive  
2  positive  
3  positive  
4  positive  
5  positive  
6  positive  
7  positive  
8  positive  
9  positive

In [7]:
'''
Tokenization & stemming

For the text processing, nltk library is used. First, 
the tweets are tokenized using nlkt.word_tokenize and then, 
stemming is done using PorterStemmer as the tweets are 100% in spanish.
The Spanish text also has some problems. Nouns that don’t need to be stemmed, 
such as ‘lugar’ (place) and ‘tiempo’ (time), have been reduced to unnecessary base forms.
In addition, verbs that come from common roots are stemmed inconsistently. 
For example, the verb ‘quiero’ (I want) is reduced to ‘quier’
, but you can see that another form of this verb, ‘queremos’ (we want) would be stemmed to ‘quer’ below.
'''
data = TwitterData_TokenStem(data)           
data.tokenize()
data.stem()

#data after tokenization & stemming
data.processed_data.head(10)

username                 date  retweets  favorites  \
0        kabrota  2017-09-24 23:59:12         1          1   
1       najarumi  2017-09-24 23:56:39         1          2   
2  MerceMrsDeath  2017-09-24 23:55:36         0          0   
3    nohopararan  2017-09-25 23:50:06         0          1   
4       OsamaPls  2017-09-25 23:45:55         0          0   
5      breifjart  2017-09-25 23:40:22         0          0   
6      MIQUEL_01  2017-09-25 23:36:39         0          2   
7        r_wicca  2017-09-25 23:33:48         0          0   
8  rafavilaclara  2017-09-26 23:57:59         0          0   
9      ateixidom  2017-09-26 23:51:27         0          0   

                                                text  \
0  [els, del, piolin, estan, que, mosseg, el, sop...   
1  [q, gràn, la, guàrdi, civil, dexpañ, internaci...   
2  [però, a, veur, k, mda, mes, manestuos, i, ele...   
3  [freepiolin, detencions, portavionsref, no, ar...   
4                       [a, jur, piolin, freepiolin]   
5  [qualsevol, dia, a, una, ciutat, despany, free...   
6  [las, urnas, las, carg, el, diabloel, dia, que...   
7  [quin, nivell, entre, el, freepiolin, i, ja, e...   
8  [es, al·lucinant, som, una, colòni, començ, a,...   
9  [insult, no, som, nosaltr, qui, hem, fletat, u...   

                                            emoticon location      mentions  \
0                                                 []      NaN           NaN   
1  ["'\\U0001f425'", "'\\U0001f425'", "'\\U0001f4...      NaN           NaN   
2                                                 []      NaN           NaN   
3                                                 []      NaN           NaN   
4                                                 []      NaN           NaN   
5                                                 []      NaN           NaN   
6                                                 []      NaN           NaN   
7  ["'\\U0001f923'", "'\\U0001f923'", "'\\U0001f9...      NaN   @albert_pla   
8                                                 []      NaN           NaN   
9                                                 []      NaN  @mariusserra   

                                             hashtag                  id  \
0                                        #FreePiolin  912104160069345280   
1                               #FreePiolin #Votarem  912103517011283968   
2                                        #FreePiolin  912103254921814016   
3                                 #freepiolin #ref1o  912464257526243330   
4                                        #FreePiolin  912463203074035712   
5         #FreePiolin #1OCTReferendum #Cataloniahttp  912461809310949376   
6  #LoQueLaUrnaEsconde #FreePiolín #HolaDictadurapic  912460873595269120   
7                                        #FreePiolin  912460157619179521   
8                                        #FreePiolin  912828629465300992   
9                                        #FreePiolin  912826985373732865   

                                           permalink language coordinates  \
0  https://twitter.com/kabrota/status/91210416006...       ca         NaN   
1  https://twitter.com/najarumi/status/9121035170...       ca         NaN   
2  https://twitter.com/MerceMrsDeath/status/91210...       ca         NaN   
3  https://twitter.com/nohopararan/status/9124642...       ca         NaN   
4  https://twitter.com/OsamaPls/status/9124632030...       ca         NaN   
5  https://twitter.com/breifjart/status/912461809...       ca         NaN   
6  https://twitter.com/MIQUEL_01/status/912460873...       ca         NaN   
7  https://twitter.com/r_wicca/status/91246015761...       ca         NaN   
8  https://twitter.com/rafavilaclara/status/91282...       ca         NaN   
9  https://twitter.com/ateixidom/status/912826985...       ca         NaN   

  sentiment                                     tokenized_text  
0  positive  [Els, del, Piolin, estan, que, mosseguen, El, ...  
1  positive  [Q, gràn, 

In [8]:
'''
Building the wordlist
The wordlist (dictionary) is build by simple count of occurences of every unique word across all of the training dataset.
Before building the final wordlist for the model, let's take a look at the non-filtered version
'''


words = Counter()
for idx in data.processed_data.index:
    words.update(data.processed_data.loc[idx, "text"])


#the most common words 
words.most_common(10)

[('votarem', 23397),
 ('de', 13911),
 ('a', 12891),
 ('la', 12777),
 ('i', 10406),
 ('que', 9313),
 ('el', 8463),
 ('freepiolin', 6812),
 ('no', 6702),
 ('oct', 5730)]

In [9]:
'''
The most common words (as expected) are the typical spanish stopwords. 
We will filter them out, however, as purpose of this analysis is to determine sentiment, 
words like "not" and "n't" can influence it greatly. Having this in mind, this word will be whitelisted.
'''

stopwords=nltk.corpus.stopwords.words("spanish")
whitelist = ['espa']
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]


#the most common words after delition of spanish stopwords
words.most_common(10)

[('votarem', 23397),
 ('i', 10406),
 ('freepiolin', 6812),
 ('oct', 5730),
 ('per', 5674),
 ('els', 4231),
 ('si', 3381),
 ('amb', 3205),
 ('cataluny', 2716),
 ('democràci', 2254)]

In [10]:
stopwords=nltk.corpus.stopwords.words("english")
whitelist = []
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]
        
print('----------------------------------------------------------------------')
print('the most common words after delition of spanish and english stopwords are:\n'+str(words.most_common(10)))
print('----------------------------------------------------------------------\n\n')

----------------------------------------------------------------------
the most common words after delition of spanish and english stopwords are:
[('votarem', 23397), ('freepiolin', 6812), ('oct', 5730), ('per', 5674), ('els', 4231), ('si', 3381), ('amb', 3205), ('cataluny', 2716), ('democràci', 2254), ('mes', 2076)]
----------------------------------------------------------------------




In [ ]:
data = TwitterData_Wordlist(data)
data.build_wordlist()

if plot_all :
    words = pd.read_csv("data/wordlist.csv")
    x_words = list(words.loc[0:7,"word"])
    x_words.reverse()
    y_occ = list(words.loc[0:7,"occurrences"])
    y_occ.reverse()

    dist = [
        graph_objs.Bar(
            x=y_occ,
            y=x_words,
            orientation="h"
    )]
    
    plotly.offline.iplot({"data":dist, "layout":graph_objs.Layout(title="Top words in built wordlist")})

In [ ]:
data = TwitterData_BagOfWords(data)
bow, labels = data.build_data_model()

print('----------------------------------------------------------------------')
print('The bow\n'+str(bow.head(10)))
print('----------------------------------------------------------------------\n\n')

In [ ]:
if plot_all :

    grouped = bow.groupby(["label"]).sum()
    words_to_visualize = []
    sentiments = ['positive','negative','neutral']
    #get the most 7 common words for every sentiment
    for sentiment in sentiments:
        words = grouped.loc[sentiment,:]
        words.sort_values(inplace=True,ascending=False)
        for w in words.index[:10]:
            if w not in words_to_visualize:
                words_to_visualize.append(w)
                
                
    #visualize it
    plot_data = []
    for sentiment in sentiments:
        plot_data.append(graph_objs.Bar(
                x = [w.split("_")[0] for w in words_to_visualize],
                y = [grouped.loc[sentiment,w] for w in words_to_visualize],
                name = sentiment
        ))
        
    plotly.offline.iplot({
            "data":plot_data,
            "layout":graph_objs.Layout(title="Most common words across sentiments")
        })

In [ ]:
def log(x):
    #can be used to write to log file
    print(x)

def test_classifier(X_train, y_train, X_test, y_test, classifier):
    if __name__=='__main__':
        log("")
        log("===============================================")
        classifier_name = str(type(classifier).__name__)
        log("Testing " + classifier_name)
        now = time()
        list_of_labels = sorted(list(set(y_train)))
        model = classifier.fit(X_train, y_train)
        log("Learing time {0}s".format(time() - now))
        now = time()
        predictions = model.predict(X_test)
        log("Predicting time {0}s".format(time() - now))

        precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
        recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
        accuracy = accuracy_score(y_test, predictions)
        f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
        log("=================== Results ===================")
        log("             Negative    Neutral    Positive")
        log("F1       " + str(f1))
        log("Precision" + str(precision))
        log("Recall   " + str(recall))
        log("Accuracy " + str(accuracy))
        log("===============================================")

        return precision, recall, accuracy, f1

def cv(classifier, X_train, y_train):
    if __name__=='__main__':
        log("===============================================")
        classifier_name = str(type(classifier).__name__)
        now = time()
        log("Crossvalidating " + classifier_name + "...")
        accuracy = [cross_val_score(classifier, X_train, y_train, cv=3, n_jobs=1)]
        log("Crosvalidation completed in {0}s".format(time() - now))
        log("Accuracy: " + str(accuracy[0]))
        log("Average accuracy: " + str(np.array(accuracy[0]).mean()))
        log("===============================================")
        return accuracy

In [ ]:
'''
Experiment 1: BOW + Naive Bayes
It is nice to see what kind of results we might get from such simple model. 
The bag-of-words representation is binary, so Naive Bayes Classifier seems 
like a nice algorithm to start the experiments.
The experiment will be based on 7:3 train:test stratified split.
'''

if do_BernoulliNB :
    X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow.iloc[:, 0],
                                                        train_size=0.7, stratify=bow.iloc[:, 0],
                                                        random_state=seed)
    precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, BernoulliNB())
    nb_acc = cv(BernoulliNB(), bow.iloc[:,1:], bow.iloc[:,0])


In [ ]:
data = TwitterData_ExtraFeatures()
data.initialize("data/train.txt")
data.build_features()
data.cleanup(TwitterCleanuper())
data.tokenize()         
data.stem()
data.build_wordlist()
data_model, labels = data.build_data_model()

if plot_all:
    sentiments = ['positive','negative']
    plots_data_ef = []
    for what in map(lambda o: "number_of_"+o,["exclamation","hashtags","question"]):
        ef_grouped = data_model[data_model[what]>=1].groupby(["label"]).count()
        plots_data_ef.append({"data":[graph_objs.Bar(
                x = sentiments,
                y = [ef_grouped.loc[s,:][0] for s in sentiments],
        )], "title":"How feature \""+what+"\" separates the tweets"})
        
    i = 0
    for plot_data_ef in plots_data_ef:
        plotly.offline.iplot({
                "data":plot_data_ef["data"],
                "layout":graph_objs.Layout(title=plot_data_ef["title"])
        })



In [ ]:
if do_RandomForest :
    X_train, X_test, y_train, y_test = train_test_split(data_model.iloc[:, 1:], data_model.iloc[:, 0],
                                                        train_size=0.7, stratify=data_model.iloc[:, 0],
                                                        random_state=seed)
    precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, RandomForestClassifier(random_state=seed,n_estimators=403,n_jobs=-1))
    rf_acc = cv(RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed),data_model.iloc[:, 1:], data_model.iloc[:, 0])

In [ ]:
word2vec = Word2VecProvider()
# REPLACE PATH TO THE FILE
word2vec.load("data/embedding_file")

In [ ]:
similarity_columns = ["fobia_similarity", "votarem_similarity", "catalexit_similarity"]
td = TwitterData()
td.initialize("data/train.txt")
td.build_features()
td.cleanup(TwitterCleanuper())
td.tokenize()
td.stem()
td.build_wordlist()
td.build_final_model(word2vec, similarity_columns= similarity_columns)

data_model = td.data_model
print(str(data_model.head(10)))

In [ ]:
data_model.drop("original_id",axis=1,inplace=True)
columns_to_plot = similarity_columns
print(str(columns_to_plot))

In [ ]:
if plot_all:
    #bad, good, info = columns_to_plot
    bad, good, info = columns_to_plot
    sentiments = ['positive','negative','neutral']


    only_positive = data_model[data_model[good]>=data_model[bad]]
    only_positive = only_positive[only_positive[good]>=only_positive[info]].groupby(["label"]).count()

    only_info = data_model[data_model[info]>=data_model[good]]
    only_info = only_info[only_info[info]>=only_info[bad]].groupby(["label"]).count()

    only_negative = data_model[data_model[bad] >= data_model[good]]
    only_negative = only_negative[only_negative[bad] >= only_negative[info]].groupby(["label"]).count()

    plot_data_w2v = []
    for sentiment in sentiments:
        plot_data_w2v.append(graph_objs.Bar(
                x = ["españa", "votarem", "catalexit"],
                y = [only_positive.loc[sentiment,:][0], only_negative.loc[sentiment,:][0], only_info.loc[sentiment,:][0]],
                name = "Number of dominating " + sentiment
        ))
        
    plotly.offline.iplot({
            "data":plot_data_w2v,
            "layout":graph_objs.Layout(title="Number of tweets dominating on similarity to: españa, votarem, catalexit")
        })

In [ ]:
'''
Experiment 3: full model + Random Forest

The model is now complete. With a lot of new features, 
the learning algorithms should perform totally differently on the new data set.
'''
if do_RandomForest_full_model :
    X_train, X_test, y_train, y_test = train_test_split(data_model.iloc[:, 1:], data_model.iloc[:, 0],
                                                        train_size=0.7, stratify=data_model.iloc[:, 0],
                                                        random_state=seed)


    precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed))
    rf_acc = cv(RandomForestClassifier(n_estimators=403,n_jobs=-1,random_state=seed),data_model.iloc[:, 1:], data_model.iloc[:, 0])

In [ ]:
'''
Experiment 4: full model + XGBoost

XGBoost is relatively new machine learning algorithm based on decision trees and boosting. 
It is highly scalable and provides results, which are often higher than those obtained using popular algorithms 
such as Random Forest or SVM.
Important: XGBoost exposes scikit-learn interface, 
but it needs to be installed as an additional python package. 
See this page to see more: https://xgboost.readthedocs.io/en/latest/build.html
'''
if do_XGBoost :
    X_train, X_test, y_train, y_test = train_test_split(data_model.iloc[:, 1:], data_model.iloc[:, 0],
                                                        train_size=0.7, stratify=data_model.iloc[:, 0],
                                                        random_state=seed)
    precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, XGBoostClassifier(seed=seed))
    xgb_acc = cv(XGBoostClassifier(seed=seed),data_model.iloc[:, 1:], data_model.iloc[:, 0])

In [ ]:
'''
Finding best parameters for XGBoost

Warning the code bellow executes for a really long time. 
'''

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            log("Model with rank: {0}".format(i))
            log("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            log("Parameters: {0}".format(results['params'][candidate]))
            log("")

def best_fit(X_train, y_train, n_iter=5):
    
    parameters = {
        "n_estimators":[103,201, 403],
        "max_depth":[3,10,15, 30],
        "objective":["multi:softmax","binary:logistic"],
        "learning_rate":[0.05, 0.1, 0.15, 0.3]
    }

    rand_search = RandomizedSearchCV(XGBoostClassifier(seed=seed),param_distributions=parameters,
                                     n_iter=n_iter,scoring="accuracy",
                                     n_jobs=-1,cv=8)

    import time as ttt
    now = time()
    log(ttt.ctime())
    rand_search.fit(X_train, y_train)
    report(rand_search.cv_results_, 10)
    log(ttt.ctime())
    log("Search took: " + str(time() - now))

best_fit(data_model.iloc[:, 1:], data_model.iloc[:, 0], n_iter=10)


In [ ]:
'''
Test data classification

After finding best cross-validated parameter for the XGBoost, 
it's time to load the test data and predict sentiment for them. 
Final classifier will be trained on the whole training set. 
'''

test_data = TwitterData()
test_data.initialize("data/test.txt", is_testing_set=True)
test_data.build_features()
test_data.cleanup(TwitterCleanuper())
test_data.tokenize()
test_data.stem()
test_data.build_wordlist()
test_data.build_final_model(word2vec)

test_data.data_model.head(5)

In [ ]:
test_model = test_data.data_model

data_model = td.data_model

xgboost = XGBoostClassifier(seed=seed,n_estimators=403,max_depth=10,objective="binary:logistic",learning_rate=0.15)
xgboost.fit(data_model.iloc[:,1:],data_model.iloc[:,0])
predictions = xgboost.predict(test_model.iloc[:,1:])

test_model.head(5)

In [ ]:
'''
Feature importance

Let's take a look at the final model feature importance.
'''

features = {}
for idx, fi in enumerate(xgboost.feature_importances_):
    features[test_model.columns[1+idx]] = fi

important = []
for f in sorted(features,key=features.get,reverse=True):
    important.append((f,features[f]))
    # print(f + " " + str(features[f]))

In [ ]:
to_show = list(filter(lambda f: not f[0].startswith("word2vec") and not f[0].endswith("_similarity"),important))[:20]
to_show.reverse()
features_importance = [
    graph_objs.Bar(
        x=[f[1] for f in to_show],
        y=[f[0] for f in to_show],
        orientation="h"
)]
plotly.offline.iplot({"data":features_importance, "layout":graph_objs.Layout(title="Most important features in the final model",
      margin=graph_objs.Margin(
           l=200,
        pad=3
    ),)})